In [ ]:
!pip install -q -U trl transformers accelerate peft datasets bitsandbytes evaluate git+https://github.com/huggingface/huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

model_name = "EMBEDDIA/sloberta"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tjasad/Slovene_SuperGLUE_BoolQ")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
def preprocess_data(examples):
    passage_inputs = [f"passage : {x} " for x in examples["passage"]]
    question_inputs = [f"question : {x} " for x in examples["question"]]
    inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

    model_inputs = tokenizer(inputs, max_length=400, truncation=True)

    model_inputs["labels"] = examples["label"]
    model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import AutoModelForSequenceClassification, set_seed


id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

In [11]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_boolq_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.644200,0.552337,0.777778,0.680556
100,0.381700,1.047496,0.555556,0.590188
150,0.035500,1.458064,0.611111,0.625514
200,0.009800,1.636425,0.611111,0.625514
250,0.006500,1.816721,0.611111,0.625514
300,0.005000,1.836287,0.611111,0.625514
350,0.004300,1.863543,0.611111,0.625514
400,0.004000,1.863270,0.611111,0.625514


TrainOutput(global_step=400, training_loss=0.1363663448393345, metrics={'train_runtime': 80.8697, 'train_samples_per_second': 39.57, 'train_steps_per_second': 4.946, 'total_flos': 396831082581360.0, 'train_loss': 0.1363663448393345, 'epoch': 33.333333333333336})

In [14]:
trainer.evaluate()

{'eval_loss': 1.8632704019546509,
 'eval_accuracy': 0.6111111111111112,
 'eval_f1': 0.6255144032921812,
 'eval_runtime': 0.1684,
 'eval_samples_per_second': 106.885,
 'eval_steps_per_second': 17.814,
 'epoch': 33.333333333333336}

In [ ]:
trainer.push_to_hub(new_model_name)

In [16]:
from transformers import pipeline

adapter_name = "tjasad/" + new_model_name

# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/886 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

[{'label': 'False', 'score': 0.9963114857673645}]